Option 2:
Converting 2 Dataframes and exploring the features for fraudulent Transactions and build a model to predict fraudulent transactions

In [25]:
import os

In [26]:
os.chdir(r'D:\ashok\mypers_proj\summinds')

In [27]:
import pandas as pd
import numpy as np

In [28]:
import json

with open('customersdata.json', encoding='latin-1') as f:
    json_str_list = f.read().split('\n')

In [29]:
json.loads(json_str_list[0])

{'fraudulent': False,
 'customer': {'customerEmail': 'josephhoward@yahoo.com',
  'customerPhone': '400-108-5415',
  'customerDevice': 'yyeiaxpltf82440jnb3v',
  'customerIPAddress': '8.129.104.40',
  'customerBillingAddress': '5493 Jones Islands\nBrownside, CA 51896'},
 'orders': [{'orderId': 'vjbdvd',
   'orderAmount': 18,
   'orderState': 'pending',
   'orderShippingAddress': '5493 Jones Islands\nBrownside, CA 51896'},
  {'orderId': 'yp6x27',
   'orderAmount': 26,
   'orderState': 'fulfilled',
   'orderShippingAddress': '5493 Jones Islands\nBrownside, CA 51896'}],
 'paymentMethods': [{'paymentMethodId': 'wt07xm68b',
   'paymentMethodRegistrationFailure': True,
   'paymentMethodType': 'card',
   'paymentMethodProvider': 'JCB 16 digit',
   'paymentMethodIssuer': 'Citizens First Banks'}],
 'transactions': [{'transactionId': 'a9lcj51r',
   'orderId': 'vjbdvd',
   'paymentMethodId': 'wt07xm68b',
   'transactionAmount': 18,
   'transactionFailed': False},
  {'transactionId': 'y4wcv03i',
   


# Creating a Seperate Dataframes for transactions,payments,customers,orders

In [30]:
customers,orders,transactions,payment_ids,fraudulent=[],[],[],[],[]
for i, json_str in enumerate(json_str_list):
    if json_str:
        data=json.loads(json_str)
        fraudulent.append(data['fraudulent'])
        customers.append(data['customer'])
        orders.append(data['orders'])
        #     print('\n')/
        transactions.append(data['transactions'])
        payment_ids.append(data['paymentMethods'])
#     except:
#         print(i)

In [31]:
len(fraudulent),len(orders),len(transactions),len(payment_ids)

(168, 168, 168, 168)

In [32]:
df_customers=pd.DataFrame()
df_orders=pd.DataFrame()
df_transactions=pd.DataFrame()
df_payments=pd.DataFrame()
for i in range(len(fraudulent)):
    li = customers[i]
    li['customer_id']=i
    li['fraudulent']=fraudulent[i]
    df_customers=df_customers.append(li,ignore_index=True)
    li=pd.DataFrame(orders[i])
    li['id']=i
    li['fraudulent']=fraudulent[i]
    df_orders=df_orders.append(li,ignore_index=True)
    li=pd.DataFrame(transactions[i])
    li['id']=i
    li['fraudulent']=fraudulent[i]
    df_transactions=df_transactions.append(li,ignore_index=True)
    li=pd.DataFrame(payment_ids[i])
    li['id']=i
    li['fraudulent']=fraudulent[i]
    df_payments=df_payments.append(li,ignore_index=True)


C:\Users\Attune\Anaconda3\lib\site-packages\pandas\core\frame.py:6692: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort)


For each of the fraudulent or non fraudulent, ids are maintained which is nothing but the customerid, so maintaining a unique column id lo link with customer related information

In [33]:
df_orders['id'].nunique(),df_payments['id'].nunique(),df_customers['customer_id'].nunique(),df_transactions['id'].nunique()

(141, 156, 168, 141)

In [34]:
df_orders.head()

,fraudulent,id,orderAmount,orderId,orderShippingAddress,orderState
0,False,0,18.0,vjbdvd,"5493 Jones Islands\nBrownside, CA 51896",pending
1,False,0,26.0,yp6x27,"5493 Jones Islands\nBrownside, CA 51896",fulfilled
2,True,1,45.0,nlghpa,"898 Henry Ports\nNew Keithview, CA 95893-2497",fulfilled
3,True,1,23.0,uw0eeb,356 Elizabeth Inlet Suite 120\nPort Joshuabury...,fulfilled
4,True,1,43.0,bn44oh,"5093 Bryan Forks\nJoshuaton, FM 01565-9801",fulfilled


In [35]:
df_transactions.head()

,fraudulent,id,orderId,paymentMethodId,transactionAmount,transactionFailed,transactionId
0,False,0,vjbdvd,wt07xm68b,18.0,False,a9lcj51r
1,False,0,yp6x27,wt07xm68b,26.0,False,y4wcv03i
2,True,1,nlghpa,41ug157xz,45.0,False,5mi94sfw
3,True,1,uw0eeb,41ug157xz,23.0,False,br8ba1nu
4,True,1,bn44oh,y3xp697jx,43.0,True,a33145ss


In [36]:
df_customers['fraudulent'].value_counts()

0.0    107
1.0     61
Name: fraudulent, dtype: int64

In [37]:
df_orders.columns,df_payments.columns,df_transactions.columns,df_customers.columns

(Index(['fraudulent', 'id', 'orderAmount', 'orderId', 'orderShippingAddress',
        'orderState'],
       dtype='object'),
 Index(['fraudulent', 'id', 'paymentMethodId', 'paymentMethodIssuer',
        'paymentMethodProvider', 'paymentMethodRegistrationFailure',
        'paymentMethodType'],
       dtype='object'),
 Index(['fraudulent', 'id', 'orderId', 'paymentMethodId', 'transactionAmount',
        'transactionFailed', 'transactionId'],
       dtype='object'),
 Index(['customerBillingAddress', 'customerDevice', 'customerEmail',
        'customerIPAddress', 'customerPhone', 'customer_id', 'fraudulent'],
       dtype='object'))

In [38]:
df_orders['orderId'].nunique(),df_transactions['orderId'].nunique()

(478, 478)

In [39]:
df_order_transactions=pd.merge(df_transactions,df_orders[['orderId','orderAmount','orderShippingAddress','orderState']],on='orderId',how='inner')

In [40]:
df_order_transactions.shape,df_orders.shape,df_transactions.shape

((623, 10), (478, 6), (623, 7))

As order and transactions are linked with orderid, combining the dataframes and joining based on orderid,

In [41]:
df_order_transactions.sort_values(by=['orderId'])

,fraudulent,id,orderId,paymentMethodId,transactionAmount,transactionFailed,transactionId,orderAmount,orderShippingAddress,orderState
82,False,19,00z9c8,bcetnywn4,29.0,False,ubm4tzca,29.0,"34944 Glenn Mountain\nSherryhaven, TN 36252",fulfilled
305,True,84,014gsw,pube8iuin,67.0,False,0zv4q3v7,67.0,"96103 Riley Walks\nMichellefort, ID 98085-6066",pending
304,True,84,014gsw,pube8iuin,67.0,True,gidcndd6,67.0,"96103 Riley Walks\nMichellefort, ID 98085-6066",pending
211,True,60,02x0ec,yym5xo9qi,15.0,False,u2mtko2f,15.0,"15587 Rios Center\nEast Dawn, AR 22196",fulfilled
285,True,80,03euof,v50sqfm8h,47.0,False,0w4ng8t4,47.0,49680 Brian Squares Apt. 122\nPort Walterburgh...,fulfilled
366,True,99,06hsk6,ru53ucaj4,25.0,False,0flz070f,25.0,"6609 Hannah Road Suite 577\nRichardchester, AL...",fulfilled
239,True,65,0b6r24,opyep1aoj,30.0,False,0ltw4chk,30.0,"11704 Andrew Villages Apt. 035\nJamesfurt, OR ...",fulfilled
238,True,65,0b6r24,opyep1aoj,30.0,True,h1zprjml,30.0,"11704 Andrew Villages Apt. 035\nJamesfurt, OR ...",fulfilled
50,False,12,0esxhp,ow9u8ogs4,16.0,False,jz6qu6zy,16.0,"355 Tamara Plains\nNorth Judyton, LA 04531",fulfilled
415,False,113,0linjx,4lbflweug,43.0,True,82m0gkqr,43.0,"4608 Travis Spring\nSarahtown, KS 65613",fulfilled


So for each of the orders, there are multiple transactions, so some transactions might get failed

In [42]:
df_order_transactions['paymentMethodId'].nunique()

237

In [43]:
df_order_transactions.shape

(623, 10)

Check whether for some transactions, order amount is not equal to transaction amount may be because of that the transaction might get effected, if so can be considered as a feature

In [44]:
np.where((df_order_transactions['orderAmount']!=df_order_transactions['transactionAmount'])>1)

(array([], dtype=int64),)

# Analysis of  Payments Dataframe

In [45]:
df_payments.head()

,fraudulent,id,paymentMethodId,paymentMethodIssuer,paymentMethodProvider,paymentMethodRegistrationFailure,paymentMethodType
0,False,0,wt07xm68b,Citizens First Banks,JCB 16 digit,True,card
1,True,1,y3xp697jx,Solace Banks,VISA 16 digit,True,bitcoin
2,True,1,6krszxc05,Vertex Bancorp,VISA 16 digit,False,card
3,True,1,5z1szj2he,His Majesty Bank Corp.,Diners Club / Carte Blanche,False,card
4,True,1,m52tx8e1s,Vertex Bancorp,Mastercard,False,card


In [46]:
print ('unique values on a whole for payment_Method Issuer :',df_payments.paymentMethodIssuer.nunique(),
       '\n unique values on a whole for payment_Method Provider :',df_payments.paymentMethodProvider.nunique(),
      '\n unique values on a whole for payment_Method Type :',df_payments.paymentMethodType.nunique(),
      '\n unique values on a whole for payment_Method Id :',df_payments.paymentMethodId.nunique(),)

unique values on a whole for payment_Method Issuer : 20 
 unique values on a whole for payment_Method Provider : 10 
 unique values on a whole for payment_Method Type : 4 
 unique values on a whole for payment_Method Id : 342


So payment Method Issuer, Provider and type of Payment can be considered as a features which can effect Fraudulent transactions

In [47]:
df_payments.shape

(342, 7)

In [48]:
df_order_transactions['paymentMethodId'].nunique()

237

In [49]:
df_order_transactions.shape

(623, 10)

In [50]:
df_order_transactions.head()

,fraudulent,id,orderId,paymentMethodId,transactionAmount,transactionFailed,transactionId,orderAmount,orderShippingAddress,orderState
0,False,0,vjbdvd,wt07xm68b,18.0,False,a9lcj51r,18.0,"5493 Jones Islands\nBrownside, CA 51896",pending
1,False,0,yp6x27,wt07xm68b,26.0,False,y4wcv03i,26.0,"5493 Jones Islands\nBrownside, CA 51896",fulfilled
2,True,1,nlghpa,41ug157xz,45.0,False,5mi94sfw,45.0,"898 Henry Ports\nNew Keithview, CA 95893-2497",fulfilled
3,True,1,uw0eeb,41ug157xz,23.0,False,br8ba1nu,23.0,356 Elizabeth Inlet Suite 120\nPort Joshuabury...,fulfilled
4,True,1,bn44oh,y3xp697jx,43.0,True,a33145ss,43.0,"5093 Bryan Forks\nJoshuaton, FM 01565-9801",fulfilled


In [51]:
df_order_trns_pymt=pd.merge(df_order_transactions,df_payments[df_payments.columns[2:]],on='paymentMethodId',how='inner')

In [52]:
df_order_trns_pymt.head()

,fraudulent,id,orderId,paymentMethodId,transactionAmount,transactionFailed,transactionId,orderAmount,orderShippingAddress,orderState,paymentMethodIssuer,paymentMethodProvider,paymentMethodRegistrationFailure,paymentMethodType
0,False,0,vjbdvd,wt07xm68b,18.0,False,a9lcj51r,18.0,"5493 Jones Islands\nBrownside, CA 51896",pending,Citizens First Banks,JCB 16 digit,True,card
1,False,0,yp6x27,wt07xm68b,26.0,False,y4wcv03i,26.0,"5493 Jones Islands\nBrownside, CA 51896",fulfilled,Citizens First Banks,JCB 16 digit,True,card
2,True,1,nlghpa,41ug157xz,45.0,False,5mi94sfw,45.0,"898 Henry Ports\nNew Keithview, CA 95893-2497",fulfilled,Bastion Banks,American Express,False,bitcoin
3,True,1,uw0eeb,41ug157xz,23.0,False,br8ba1nu,23.0,356 Elizabeth Inlet Suite 120\nPort Joshuabury...,fulfilled,Bastion Banks,American Express,False,bitcoin
4,True,1,bn44oh,y3xp697jx,43.0,True,a33145ss,43.0,"5093 Bryan Forks\nJoshuaton, FM 01565-9801",fulfilled,Solace Banks,VISA 16 digit,True,bitcoin


Only customer related information is left out which we can get from joining id with customer id

In [53]:
df_customers['customer_id']=df_customers['customer_id'].astype(int)

In [54]:
df_customers.head(5)

,customerBillingAddress,customerDevice,customerEmail,customerIPAddress,customerPhone,customer_id,fraudulent
0,"5493 Jones Islands\nBrownside, CA 51896",yyeiaxpltf82440jnb3v,josephhoward@yahoo.com,8.129.104.40,400-108-5415,0,0.0
1,356 Elizabeth Inlet Suite 120\nPort Joshuabury...,r0jpm7xaeqqa3kr6mzum,evansjeffery@yahoo.com,219.173.211.202,1-788-091-7546,1,1.0
2,"8478 Sean Ridges Apt. 441\nDavisberg, PR 72250",4m7h5ipl1shyavt6vv2r,andersonwilliam@yahoo.com,67b7:3db8:67e0:3bea:b9d0:90c1:2b60:b9f0,024.420.0375,2,0.0
3,"7769 Elizabeth Bridge Apt. 343\nNortonstad, FM...",slovx60t0i558may4ks0,rubenjuarez@yahoo.com,95de:8565:5a66:792c:26e0:6cfb:7d87:11af,670.664.8168x94985,3,0.0
4,"148 Russell Lodge Apt. 445\nPort Jenniferside,...",j0pd24k5h8dl2fqu0cz4,uchen@malone.com,196.89.235.192,1-981-877-0870,4,1.0


In [55]:
df_comb=pd.merge(df_order_trns_pymt,df_customers[df_customers.columns[:-1]],left_on='id',right_on='customer_id',how='inner')

In [59]:
df_comb.head()

,fraudulent,id,orderId,paymentMethodId,transactionAmount,transactionFailed,transactionId,orderAmount,orderShippingAddress,orderState,paymentMethodIssuer,paymentMethodProvider,paymentMethodRegistrationFailure,paymentMethodType,customerBillingAddress,customerDevice,customerEmail,customerIPAddress,customerPhone,customer_id
0,False,0,vjbdvd,wt07xm68b,18.0,False,a9lcj51r,18.0,"5493 Jones Islands\nBrownside, CA 51896",pending,Citizens First Banks,JCB 16 digit,True,card,"5493 Jones Islands\nBrownside, CA 51896",yyeiaxpltf82440jnb3v,josephhoward@yahoo.com,8.129.104.40,400-108-5415,0
1,False,0,yp6x27,wt07xm68b,26.0,False,y4wcv03i,26.0,"5493 Jones Islands\nBrownside, CA 51896",fulfilled,Citizens First Banks,JCB 16 digit,True,card,"5493 Jones Islands\nBrownside, CA 51896",yyeiaxpltf82440jnb3v,josephhoward@yahoo.com,8.129.104.40,400-108-5415,0
2,True,1,nlghpa,41ug157xz,45.0,False,5mi94sfw,45.0,"898 Henry Ports\nNew Keithview, CA 95893-2497",fulfilled,Bastion Banks,American Express,False,bitcoin,356 Elizabeth Inlet Suite 120\nPort Joshuabury...,r0jpm7xaeqqa3kr6mzum,evansjeffery@yahoo.com,219.173.211.202,1-788-091-7546,1
3,True,1,uw0eeb,41ug157xz,23.0,False,br8ba1nu,23.0,356 Elizabeth Inlet Suite 120\nPort Joshuabury...,fulfilled,Bastion Banks,American Express,False,bitcoin,356 Elizabeth Inlet Suite 120\nPort Joshuabury...,r0jpm7xaeqqa3kr6mzum,evansjeffery@yahoo.com,219.173.211.202,1-788-091-7546,1
4,True,1,bn44oh,y3xp697jx,43.0,True,a33145ss,43.0,"5093 Bryan Forks\nJoshuaton, FM 01565-9801",fulfilled,Solace Banks,VISA 16 digit,True,bitcoin,356 Elizabeth Inlet Suite 120\nPort Joshuabury...,r0jpm7xaeqqa3kr6mzum,evansjeffery@yahoo.com,219.173.211.202,1-788-091-7546,1


In [60]:
# drop transaction amount as order amount and transaction amount are same
df_comb=df_comb.drop(columns=['transactionAmount'],axis=1)

In [61]:
# removing all the columns which are ids id,customer_id,orderid,paymentMethodid,transactionid

In [62]:
df_comb=df_comb.drop(columns=['id','customer_id','transactionId','orderId','paymentMethodId'],axis=1)

In [63]:
df_comb.head()

,fraudulent,transactionFailed,orderAmount,orderShippingAddress,orderState,paymentMethodIssuer,paymentMethodProvider,paymentMethodRegistrationFailure,paymentMethodType,customerBillingAddress,customerDevice,customerEmail,customerIPAddress,customerPhone
0,False,False,18.0,"5493 Jones Islands\nBrownside, CA 51896",pending,Citizens First Banks,JCB 16 digit,True,card,"5493 Jones Islands\nBrownside, CA 51896",yyeiaxpltf82440jnb3v,josephhoward@yahoo.com,8.129.104.40,400-108-5415
1,False,False,26.0,"5493 Jones Islands\nBrownside, CA 51896",fulfilled,Citizens First Banks,JCB 16 digit,True,card,"5493 Jones Islands\nBrownside, CA 51896",yyeiaxpltf82440jnb3v,josephhoward@yahoo.com,8.129.104.40,400-108-5415
2,True,False,45.0,"898 Henry Ports\nNew Keithview, CA 95893-2497",fulfilled,Bastion Banks,American Express,False,bitcoin,356 Elizabeth Inlet Suite 120\nPort Joshuabury...,r0jpm7xaeqqa3kr6mzum,evansjeffery@yahoo.com,219.173.211.202,1-788-091-7546
3,True,False,23.0,356 Elizabeth Inlet Suite 120\nPort Joshuabury...,fulfilled,Bastion Banks,American Express,False,bitcoin,356 Elizabeth Inlet Suite 120\nPort Joshuabury...,r0jpm7xaeqqa3kr6mzum,evansjeffery@yahoo.com,219.173.211.202,1-788-091-7546
4,True,True,43.0,"5093 Bryan Forks\nJoshuaton, FM 01565-9801",fulfilled,Solace Banks,VISA 16 digit,True,bitcoin,356 Elizabeth Inlet Suite 120\nPort Joshuabury...,r0jpm7xaeqqa3kr6mzum,evansjeffery@yahoo.com,219.173.211.202,1-788-091-7546


In [64]:
df_comb['fraudulent']=df_comb['fraudulent'].apply(lambda x:1 if x else 0 )
df_comb['transactionFailed']=df_comb['transactionFailed'].apply(lambda x:1 if x else 0 )
df_comb['paymentMethodRegistrationFailure']=df_comb['paymentMethodRegistrationFailure'].apply(lambda x:1 if x else 0 )

In [65]:
df_comb

,fraudulent,transactionFailed,orderAmount,orderShippingAddress,orderState,paymentMethodIssuer,paymentMethodProvider,paymentMethodRegistrationFailure,paymentMethodType,customerBillingAddress,customerDevice,customerEmail,customerIPAddress,customerPhone
0,0,0,18.0,"5493 Jones Islands\nBrownside, CA 51896",pending,Citizens First Banks,JCB 16 digit,1,card,"5493 Jones Islands\nBrownside, CA 51896",yyeiaxpltf82440jnb3v,josephhoward@yahoo.com,8.129.104.40,400-108-5415
1,0,0,26.0,"5493 Jones Islands\nBrownside, CA 51896",fulfilled,Citizens First Banks,JCB 16 digit,1,card,"5493 Jones Islands\nBrownside, CA 51896",yyeiaxpltf82440jnb3v,josephhoward@yahoo.com,8.129.104.40,400-108-5415
2,1,0,45.0,"898 Henry Ports\nNew Keithview, CA 95893-2497",fulfilled,Bastion Banks,American Express,0,bitcoin,356 Elizabeth Inlet Suite 120\nPort Joshuabury...,r0jpm7xaeqqa3kr6mzum,evansjeffery@yahoo.com,219.173.211.202,1-788-091-7546
3,1,0,23.0,356 Elizabeth Inlet Suite 120\nPort Joshuabury...,fulfilled,Bastion Banks,American Express,0,bitcoin,356 Elizabeth Inlet Suite 120\nPort Joshuabury...,r0jpm7xaeqqa3kr6mzum,evansjeffery@yahoo.com,219.173.211.202,1-788-091-7546
4,1,1,43.0,"5093 Bryan Forks\nJoshuaton, FM 01565-9801",fulfilled,Solace Banks,VISA 16 digit,1,bitcoin,356 Elizabeth Inlet Suite 120\nPort Joshuabury...,r0jpm7xaeqqa3kr6mzum,evansjeffery@yahoo.com,219.173.211.202,1-788-091-7546
5,0,0,33.0,"8478 Sean Ridges Apt. 441\nDavisberg, PR 72250",failed,Bastion Banks,JCB 16 digit,0,card,"8478 Sean Ridges Apt. 441\nDavisberg, PR 72250",4m7h5ipl1shyavt6vv2r,andersonwilliam@yahoo.com,67b7:3db8:67e0:3bea:b9d0:90c1:2b60:b9f0,024.420.0375
6,0,1,24.0,"8478 Sean Ridges Apt. 441\nDavisberg, PR 72250",fulfilled,Bastion Banks,JCB 16 digit,0,card,"8478 Sean Ridges Apt. 441\nDavisberg, PR 72250",4m7h5ipl1shyavt6vv2r,andersonwilliam@yahoo.com,67b7:3db8:67e0:3bea:b9d0:90c1:2b60:b9f0,024.420.0375
7,0,0,24.0,"8478 Sean Ridges Apt. 441\nDavisberg, PR 72250",fulfilled,Bastion Banks,JCB 16 digit,0,card,"8478 Sean Ridges Apt. 441\nDavisberg, PR 72250",4m7h5ipl1shyavt6vv2r,andersonwilliam@yahoo.com,67b7:3db8:67e0:3bea:b9d0:90c1:2b60:b9f0,024.420.0375
8,0,0,25.0,"22396 Victor Lights\nCarsonstad, MO 93914",fulfilled,Bastion Banks,JCB 16 digit,0,card,"8478 Sean Ridges Apt. 441\nDavisberg, PR 72250",4m7h5ipl1shyavt6vv2r,andersonwilliam@yahoo.com,67b7:3db8:67e0:3bea:b9d0:90c1:2b60:b9f0,024.420.0375
9,0,1,25.0,"22396 Victor Lights\nCarsonstad, MO 93914",fulfilled,His Majesty Bank Corp.,Discover,0,card,"8478 Sean Ridges Apt. 441\nDavisberg, PR 72250",4m7h5ipl1shyavt6vv2r,andersonwilliam@yahoo.com,67b7:3db8:67e0:3bea:b9d0:90c1:2b60:b9f0,024.420.0375


Order State, PaymentMethodIssuer,PaymentMethodProvider,payment Method Type are the categorical variables convert them to categorical

In [66]:
df_comb[['orderState','paymentMethodIssuer','paymentMethodProvider','paymentMethodType']].apply(pd.Series.value_counts)

,orderState,paymentMethodIssuer,paymentMethodProvider,paymentMethodType
American Express,NaN,NaN,45.0,NaN
B,NaN,10.0,NaN,NaN
Bastion Banks,NaN,58.0,NaN,NaN
Bulwark Trust Corp.,NaN,59.0,NaN,NaN
Citizens First Banks,NaN,55.0,NaN,NaN
Diners Club / Carte Blanche,NaN,NaN,58.0,NaN
Discover,NaN,NaN,41.0,NaN
Fountain Financial Inc.,NaN,60.0,NaN,NaN
Grand Credit Corporation,NaN,55.0,NaN,NaN
Her Majesty Trust,NaN,66.0,NaN,NaN


In [67]:
print(df_comb.orderState.value_counts(),
      df_comb.paymentMethodIssuer.value_counts(),
      df_comb.paymentMethodProvider.value_counts(),
      df_comb.paymentMethodType.value_counts())

fulfilled    516
failed        63
pending       44
Name: orderState, dtype: int64 Her Majesty Trust           66
His Majesty Bank Corp.      65
Vertex Bancorp              63
Fountain Financial Inc.     60
Rose Bancshares             59
Bulwark Trust Corp.         59
Bastion Banks               58
Grand Credit Corporation    55
Citizens First Banks        55
Solace Banks                41
B                           10
c                            7
e                            7
r                            6
x                            4
a                            4
o                            2
n                            1
p                            1
Name: paymentMethodIssuer, dtype: int64 JCB 16 digit                   118
VISA 16 digit                   96
Maestro                         69
Voyager                         61
Diners Club / Carte Blanche     58
VISA 13 digit                   58
JCB 15 digit                    45
American Express                45
Discover 

In [68]:
df_comb_cat=pd.get_dummies(df_comb,columns=['orderState',
                                            'paymentMethodIssuer',
                                            'paymentMethodProvider',
                                            'paymentMethodType'])

In [70]:
import seaborn as sns
df_comb_cat.corr().iloc[0,:]

fraudulent                                           1.000000
transactionFailed                                   -0.068354
orderAmount                                          0.325534
paymentMethodRegistrationFailure                     0.030605
orderState_failed                                    0.010936
orderState_fulfilled                                 0.044432
orderState_pending                                  -0.078280
paymentMethodIssuer_B                                0.152421
paymentMethodIssuer_Bastion Banks                   -0.010393
paymentMethodIssuer_Bulwark Trust Corp.              0.085314
paymentMethodIssuer_Citizens First Banks             0.072536
paymentMethodIssuer_Fountain Financial Inc.          0.013802
paymentMethodIssuer_Grand Credit Corporation        -0.099856
paymentMethodIssuer_Her Majesty Trust               -0.182505
paymentMethodIssuer_His Majesty Bank Corp.          -0.062011
paymentMethodIssuer_Rose Bancshares                  0.040771
paymentM

From the above correlation of the features with fraudulent target variable, you can see that payment method provider, issuer , transaction amount are 
impacting the transaction whether it is fraudulent or not

In [71]:
df_comb_cat.shape
df_comb_cat.columns

Index(['fraudulent', 'transactionFailed', 'orderAmount',
       'orderShippingAddress', 'paymentMethodRegistrationFailure',
       'customerBillingAddress', 'customerDevice', 'customerEmail',
       'customerIPAddress', 'customerPhone', 'orderState_failed',
       'orderState_fulfilled', 'orderState_pending', 'paymentMethodIssuer_B',
       'paymentMethodIssuer_Bastion Banks',
       'paymentMethodIssuer_Bulwark Trust Corp.',
       'paymentMethodIssuer_Citizens First Banks',
       'paymentMethodIssuer_Fountain Financial Inc.',
       'paymentMethodIssuer_Grand Credit Corporation',
       'paymentMethodIssuer_Her Majesty Trust',
       'paymentMethodIssuer_His Majesty Bank Corp.',
       'paymentMethodIssuer_Rose Bancshares',
       'paymentMethodIssuer_Solace Banks',
       'paymentMethodIssuer_Vertex Bancorp', 'paymentMethodIssuer_a',
       'paymentMethodIssuer_c', 'paymentMethodIssuer_e',
       'paymentMethodIssuer_n', 'paymentMethodIssuer_o',
       'paymentMethodIssuer_p', 'pay

In [85]:
df_customers.columns,df_customers.shape

(Index(['customerBillingAddress', 'customerDevice', 'customerEmail',
        'customerIPAddress', 'customerPhone', 'customer_id', 'fraudulent'],
       dtype='object'), (168, 7))

In [86]:
df_customers.sample(5)

,customerBillingAddress,customerDevice,customerEmail,customerIPAddress,customerPhone,customer_id,fraudulent
156,"904 James Route Suite 595\nLake Ruth, CO 66976",felexq5xniffmhhlqdqm,stevehernandez@gmail.com,601d:b1f3:1a48:e0ef:1452:906f:21dc:b526,716-379-6345,156,0.0
70,"08645 Alvarado Way Suite 514\nMillerfurt, AL 8...",132m3ourosp5a28ksvou,brockmatthew@hotmail.com,188.74.171.130,800.814.4655x012,70,0.0
62,"872 Ashley Island Apt. 254\nEast Anthonytown, ...",e3prs7zh4ueieiwlkud7,hj8maoy@1jcfcxs7,769a:4538:2bd2:8213:85ed:ca71:1ea3:dcd9,03115487766,62,1.0
20,"238 Reeves Villages\nLeslieton, SD 12318-4476",qx6nhdabeaj329zdri0o,psantiago@nelson.net,81.13.103.118,885.682.0863,20,0.0
59,585 Veronica Loop Suite 856\nEast Rebeccachest...,8fapmchbpcdesx94tnmb,cookbenjamin@hotmail.com,82.3.174.184,(287)694-5221x45370,59,0.0


In [84]:
df_customers.customerEmail.nunique()

161

In [87]:
df_customers.customerDevice.nunique()

168

From Customer Billing Address pincode and street or city can be considered as additional features,for now ignoring the customer features

In [72]:
df_non_customer=df_comb_cat.drop( ['orderShippingAddress','customerBillingAddress', 'customerDevice', 'customerEmail','customerIPAddress', 'customerPhone'],axis=1)

In [73]:
from sklearn.model_selection import   train_test_split
x_train,y_train,x_test,y_test=train_test_split(df_non_customer.drop(['fraudulent'],axis=1),df_non_customer['fraudulent'],stratify=df_non_customer['fraudulent'],test_size=0.3)

In [74]:
y_test.value_counts()

0    110
1     77
Name: fraudulent, dtype: int64

In [75]:
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LogisticRegression
from xgboost import XGBClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn import metrics
from sklearn.svm import SVC
import time
def results_mod(train_feat,test_feat,target_train,target_test):
    start=time.time()
    print('Linear SVC')
    tuned_param=({'C':[0.001,0.01,0.1,1,10]})
    svm_model=GridSearchCV(SVC(),tuned_param)
    svm_model.fit(train_feat,target_train)
    pred=svm_model.predict(test_feat)
    acc_svm=metrics.accuracy_score(target_test,pred)
    auc_svm=metrics.roc_auc_score(target_test,pred)
    conf_svm=metrics.confusion_matrix(target_test,pred)
    print('Logistic ')
    tuned_param=({'C':[0.001,0.01,0.1,1,10]})
    log_model=GridSearchCV(LogisticRegression(),tuned_param)
    log_model.fit(train_feat,target_train)
    pred=log_model.predict(test_feat)
    acc_log=metrics.accuracy_score(target_test,pred)
    auc_log=metrics.roc_auc_score(target_test,pred)
    conf_log=metrics.confusion_matrix(target_test,pred)
    tuned_param=({'max_depth':[5,10,20,50],
             'n_estimators':[10,50,100]})
    print('Random ')
    rf_model=GridSearchCV(RandomForestClassifier(),tuned_param)
    rf_model.fit(train_feat,target_train)
    pred_rf=rf_model.predict(test_feat)
    acc_rf=metrics.accuracy_score(target_test,pred_rf)
    auc_rf=metrics.roc_auc_score(target_test,pred_rf)
    conf_rf=metrics.confusion_matrix(target_test,pred_rf)
    bases = [i for i in range(1,25,5)]
    depth = [i for i in range(1,30,5)]
    lrs = [0.01,0.5,0.1]
    print('XGB ')
    tuned_param = {'max_depth':depth,'n_estimators':bases, 'learning_rate':lrs}
    xgb_model = GridSearchCV(estimator = XGBClassifier(booster='gbtree' ), param_grid=tuned_param , cv = 3, n_jobs = 4)
    xgb_model.fit(train_feat,target_train)
    pred_xgb=xgb_model.predict(test_feat)
    acc_xgb=metrics.accuracy_score(target_test,pred_xgb)
    auc_xgb=metrics.roc_auc_score(target_test,pred_xgb)
    conf_xgb=metrics.confusion_matrix(target_test,pred_xgb)
    results={'model':['svm','logistic','random forest','xgb model'],
               'accuracy':[acc_svm,acc_log,acc_rf,acc_xgb],
               'auc':[auc_svm,auc_log,auc_rf,auc_xgb],
               'confusion matrix':[conf_svm,conf_log,conf_rf,conf_xgb]}
    results_df=pd.DataFrame(results)
    end = time.time()
    time_taken = end - start
    print(time_taken)
    return results_df,svm_model,log_model,rf_model,xgb_model

In [76]:
results,model_svm,model_log,model_rf,model_xgb=results_mod(x_train,y_train,x_test,y_test)


Linear SVC


C:\Users\Attune\Anaconda3\lib\site-packages\sklearn\model_selection\_split.py:2053: FutureWarning: You should specify a value for 'cv' instead of relying on the default value. The default value will change from 3 to 5 in version 0.22.
  warnings.warn(CV_WARNING, FutureWarning)
C:\Users\Attune\Anaconda3\lib\site-packages\sklearn\svm\base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
C:\Users\Attune\Anaconda3\lib\site-packages\sklearn\svm\base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
C:\Users\Attune\Anaconda3\lib\site-packages\sklearn\svm\base.py:196: FutureWarning: The default value of g

Logistic 


C:\Users\Attune\Anaconda3\lib\site-packages\sklearn\model_selection\_split.py:2053: FutureWarning: You should specify a value for 'cv' instead of relying on the default value. The default value will change from 3 to 5 in version 0.22.
  warnings.warn(CV_WARNING, FutureWarning)
C:\Users\Attune\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\Attune\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\Attune\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\Attune\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Defa

Random 
XGB 
19.346721410751343


In [77]:
print(results)

           model  accuracy       auc       confusion matrix
0            svm  0.737968  0.714935   [[93, 17], [32, 45]]
1       logistic  0.705882  0.689610   [[86, 24], [31, 46]]
2  random forest  0.764706  0.727922   [[103, 7], [37, 40]]
3      xgb model  0.812834  0.792208  [[100, 10], [25, 52]]


Random Forest model gave better accuracy and auc

In [78]:
model_rf.best_estimator_.feature_importances_

array([0.02130951, 0.37880055, 0.012614  , 0.01435821, 0.00951403,
       0.00913251, 0.01927212, 0.01117578, 0.02153405, 0.01914765,
       0.01291452, 0.02734601, 0.05841751, 0.02042157, 0.01780748,
       0.00636642, 0.02324772, 0.00839214, 0.01067831, 0.02947363,
       0.        , 0.00098376, 0.002042  , 0.02381087, 0.00908953,
       0.01710146, 0.01358459, 0.00866945, 0.02244106, 0.01534424,
       0.01413289, 0.01622343, 0.01617128, 0.02065043, 0.0184093 ,
       0.014226  , 0.0095789 , 0.01738745, 0.02822962])

In [79]:
feats = {} # a dict to hold feature_name: feature_importance
for feature, importance in zip(x_train.columns, model_rf.best_estimator_.feature_importances_):
    feats[feature] = importance #add the name/value pair 

In [80]:
# top 5 features which are contributing to fraudulent Transactions
sorted(feats.items(), key=lambda x: x[1], reverse=True)[:5]

[('orderAmount', 0.378800552716944),
 ('paymentMethodIssuer_Her Majesty Trust', 0.05841750670424359),
 ('paymentMethodIssuer_e', 0.0294736270988208),
 ('paymentMethodType_paypal', 0.02822961578610863),
 ('paymentMethodIssuer_Grand Credit Corporation', 0.02734601220921854)]

In [81]:
# top 5 features which are not contributing to fraudulent Transactions
sorted(feats.items(), key=lambda x: x[1])[:5]

[('paymentMethodIssuer_n', 0.0),
 ('paymentMethodIssuer_o', 0.000983764025067192),
 ('paymentMethodIssuer_p', 0.0020420020926074766),
 ('paymentMethodIssuer_Solace Banks', 0.006366421147681568),
 ('paymentMethodIssuer_a', 0.008392143342766792)]